# Tree detection with DeepForest
In this notebook, we reproduce the results of tree detection step of the pipeline

In [1]:
from typing import Tuple, Generator
import os
from math import ceil

import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
from deepforest import deepforest

Using TensorFlow backend.


Define class for extracting patches from large RGB images

In [2]:
class PatchGenerator:
    def __init__(self, image: np.ndarray, patch_shape: Tuple[int, int]) -> None:
        self.image = image
        self.height, self.width, self.num_channels = self.image.shape
        self.patch_shape = patch_shape
        self._extract_patches()

    def __call__(self) -> Generator[np.ndarray, None, None]:
        for idx in range(self._calculate_num_patches()):
            yield self.patches[idx, :, :, :]

    def get_patch(self, idx: int) -> np.ndarray:
        return self.patches[idx, :, :, :]

    def get_patch_tl_br(self, idx: int) -> Tuple[int, int, int, int]:
        i, j = self._calculate_patch_coord(idx)
        v_start, v_end = i * self.patch_shape[0], (i + 1) * self.patch_shape[0]
        u_start, u_end = j * self.patch_shape[1], (j + 1) * self.patch_shape[1]
        return u_start, v_start, u_end, v_end

    def get_num_patches(self) -> int:
        return self._calculate_num_patches()

    def stitch_patches(self, patches: np.ndarray = None) -> np.ndarray:
        if patches is None:
            patches = self.patches
        stiched_image = np.zeros_like(self.image)
        for idx in range(self._calculate_num_patches()):
            u_start, v_start, u_end, v_end = self.get_patch_tl_br(idx)
            patch_shape = stiched_image[v_start:v_end, u_start:u_end, :].shape
            stiched_image[v_start:v_end, u_start:u_end, :] = patches[idx, :patch_shape[0], :patch_shape[1], :]
        return stiched_image

    def _extract_patches(self) -> np.ndarray:
        num_patches = self._calculate_num_patches()
        self.patches = np.zeros((num_patches, *self.patch_shape, self.num_channels))

        for idx in range(num_patches):
            u_start, v_start, u_end, v_end = self.get_patch_tl_br(idx)
            patch = self.image[v_start:v_end, u_start:u_end, :]
            patch_height, patch_width, _ = patch.shape
            self.patches[idx, :patch_height, :patch_width, :] = patch

    def _calculate_num_rows_columns(self) -> Tuple[int, int]:
        num_rows = ceil(self.height / self.patch_shape[0])
        num_columns = ceil(self.width / self.patch_shape[1])
        return num_rows, num_columns

    def _calculate_patch_coord(self, idx: int) -> Tuple[int, int]:
        _, num_columns = self._calculate_num_rows_columns()
        row_idx = idx // num_columns
        col_idx = idx % num_columns
        return row_idx, col_idx

    def _calculate_num_patches(self) -> int:
        num_rows, num_columns = self._calculate_num_rows_columns()
        return num_rows * num_columns


Define method for loading images

In [3]:
def load_image(image_filepath: str) -> np.ndarray:
    return cv2.imread(image_filepath)

## Use deepforest model to predict bounding boxes of tree crowns

In [6]:
MODEL_FILEPATH = "../models/final_model_4000_epochs_35.h5"
PREDICTION_OUTPUT_DIR = "../data/processed/predicted_bbox"
PATCH_SHAPE = (4000, 4000)
SCORE_THRESHOLD = 0.1

Load model

In [7]:
model = deepforest.deepforest(saved_model=MODEL_FILEPATH)

W0108 16:21:11.025869 140715621279552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Reading config file: /usr/local/lib/python3.6/dist-packages/deepforest/data/deepforest_config.yml
Loading saved model


W0108 16:21:13.271842 140715621279552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/deepforest/keras_retinanet/backend/tensorflow_backend.py:104: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0108 16:21:17.094801 140715621279552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_1:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_2:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_3:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_4:0' shape=(9, 4) dtype=float32> anchors


Define function to predict boudning boxes for given orthomosaic (RGB image)

In [8]:
def predict_bounding_boxes(orthomosaic_filepath):
    orthomosaic = load_image(orthomosaic_filepath)
    patch_generator = PatchGenerator(orthomosaic, PATCH_SHAPE)
    prediction: pd.DataFrame = None
    for idx, patch in enumerate(patch_generator()):
        pred = model.predict_image(
            numpy_image=patch, return_plot=False, score_threshold=SCORE_THRESHOLD
        )
        # Transform bounding box cooords from patch to orthomosaic
        vmin, umin, _, _ = patch_generator.get_patch_tl_br(idx)
        pred['xmin'] = pred['xmin'] + vmin
        pred['xmax'] = pred['xmax'] + vmin
        pred['ymin'] = pred['ymin'] + umin
        pred['ymax'] = pred['ymax'] + umin
        prediction = pd.concat([prediction, pred], ignore_index=True)
    return prediction

Define function to save predictions

In [9]:
def save_predictions(prediction: pd.DataFrame, orthomosaic_filepath):
    orthomosaic_filename = os.path.splitext(os.path.basename(orthomosaic_filepath))[0]
    output_filename = os.path.join(PREDICTION_OUTPUT_DIR, f"{orthomosaic_filename}_predicted_bounding_boxes.csv")
    prediction.to_csv(output_filename)

Main

In [12]:
orthomosaic_filepaths = [
    "../data/raw/Carlos Vera Arteaga RGB.tif",
    "../data/raw/Carlos Vera Guevara RGB.tif",
    "../data/raw/Flora Pluas RGB.tif",
    "../data/raw/Leonor Aspiazu RGB.tif",
    "../data/raw/Manuel Macias RGB.tif",
    "../data/raw/Nestor Macias RGB.tif",
]

for orthomosaic_filepath in tqdm(orthomosaic_filepaths):
    predictions = predict_bounding_boxes(orthomosaic_filepath)
    print(predictions.head())
    save_predictions(predictions, orthomosaic_filepath)

 17%|█▋        | 1/6 [00:17<01:29, 17.81s/it]

          xmin         ymin         xmax         ymax     score label
0  3472.602783  2084.359863  3740.785400  2330.677002  0.626113  Tree
1  2925.480225  2230.653076  3442.990723  2764.071289  0.579985  Tree
2  2368.995117  3021.250244  2828.616699  3481.844727  0.508287  Tree
3  1431.577759  3564.384033  2030.847412  3988.493652  0.452210  Tree
4  2025.169922  3700.907227  2357.979004  3998.964111  0.441826  Tree


 33%|███▎      | 2/6 [00:32<01:04, 16.02s/it]

          xmin         ymin         xmax         ymax     score label
0  2628.322021  3071.776367  2872.077393  3334.477783  0.602471  Tree
1  1783.682373  3419.016113  2029.980103  3655.197998  0.581091  Tree
2  2118.699463  3828.580811  2354.572021  3998.114502  0.558561  Tree
3  3796.686279  2701.232666  3997.942871  3050.047607  0.538558  Tree
4  2027.906372  2961.632324  2254.148438  3190.377197  0.523431  Tree


 50%|█████     | 3/6 [00:52<00:53, 17.79s/it]

...

          xmin         ymin         xmax         ymax     score label
0  3259.540283  3475.117676  3614.737549  3921.431641  0.432400  Tree
1  2252.078613  3129.029785  3099.139648  3964.230713  0.409663  Tree
2  3742.937988  3458.573242  3990.471191  3994.394531  0.333250  Tree
3  1659.075317  3775.961914  2138.611816  3995.636963  0.320874  Tree
4  3509.222412  3121.702637  3918.738037  3519.722656  0.286405  Tree


 83%|████████▎ | 5/6 [01:23<00:16, 16.04s/it]

          xmin         ymin         xmax         ymax     score label
0  3142.790771  1472.712280  3974.216553  2416.628418  0.672816  Tree
1  1724.132935  1950.795898  2367.895996  2568.429199  0.600508  Tree
2    27.317541  1729.263306  1201.323730  2865.305664  0.552456  Tree
3  2949.085693  2396.412842  3516.715576  2934.051270  0.518247  Tree
4  2397.747070  3601.624756  2900.901367  3994.374756  0.507315  Tree


100%|██████████| 6/6 [01:39<00:00, 16.55s/it]

          xmin         ymin         xmax         ymax     score label
0  3632.374512  3633.669189  3994.532471  4000.000000  0.755147  Tree
1  3137.673584  3205.398438  3501.337891  3626.599121  0.680561  Tree
2  2953.614746  3629.650146  3325.656738  3995.960938  0.622315  Tree
3  3611.481934  2435.647949  3986.134033  2857.687744  0.618454  Tree
4  3353.430664  3779.258789  3622.084961  4000.000000  0.607611  Tree


Define function to generate visualizations

In [16]:
ORTHOMOSAICS_DIR = "../data/raw/"
VISUALIZATION_OUTPUT_DIR = "../data/processed/predicted_bbox_visualizations"
COLOR = (0, 0, 255)
THICKNESS = 20

In [17]:
def visualize_bounding_boxes(orthomosaic_name):
    # Load deepforest predictions
    prediction_filename = f"{orthomosaic_name}_predicted_bounding_boxes.csv"
    prediction_filepath = os.path.join(PREDICTION_OUTPUT_DIR, prediction_filename)
    predictions = pd.read_csv(prediction_filepath)
    # Load RGB orthomosaic
    ortho_filename = f"{orthomosaic_name}.tif"
    ortho_filepath = os.path.join(ORTHOMOSAICS_DIR, ortho_filename)
    image = load_image(ortho_filepath)
    # Draw bounding boxes
    for _, row in predictions.iterrows():
        p1 = (int(row.xmin), int(row.ymin))
        p2 = (int(row.xmax), int(row.ymax))
        image = cv2.rectangle(image, p1, p2, COLOR, THICKNESS)
    # Save visualizations
    output_filename = f"{orthomosaic_name}_predicted_boxes.png"
    output_filepath = os.path.join(VISUALIZATION_OUTPUT_DIR, output_filename)
    cv2.imwrite(output_filepath, image)
    image_compressed = cv2.resize(image, None, fx=0.1, fy=0.1)
    output_filename = f"{VISUALIZATION_OUTPUT_DIR}_predicted_boxes_compressed.png"
    output_filepath = os.path.join(VISUALIZATION_OUTPUT_DIR, output_filename)
    cv2.imwrite(output_filepath, image_compressed)

Draw and save visualizations

In [18]:
orthomosaic_names = [
    "Carlos Vera Arteaga RGB",
    "Carlos Vera Guevara RGB",
    "Flora Pluas RGB",
    "Leonor Aspiazu RGB",
    "Manuel Macias RGB",
    "Nestor Macias RGB",
]
for ortho_name in tqdm(orthomosaic_names):
    visualize_bounding_boxes(ortho_name)

100%|██████████| 6/6 [00:50<00:00,  8.48s/it]
